# Yomitoku Pro Document Analyzer in SageMaker or CloudFormation

## Before use
Please deploy the service in advance using **CloudFormation** or **SageMaker Console**.

---

### Option 1 — Using SageMaker Console
1. Go to **AWS Marketplace** → Find your subscribed product → Click **Configure**  
2. Select **SageMaker Console** as the launch method  
3. In SageMaker console, choose:  
   - **Model name** (any name)  
   - **Instance type** (e.g. `ml.m5.xlarge`)  
   - **Endpoint name** (name for the inference endpoint)  
4. Click **Create endpoint** → Wait until status becomes **InService**  

---

### Option 2 — Using CloudFormation
1. Go to **AWS Marketplace** product page → **Configure** → Select **CloudFormation**  
2. Click the provided **Launch Stack** link  
3. In the CloudFormation page:  
   - Set **Stack name** (any name)  
   - Confirm parameters (instance type, etc.)  
4. Click **Create stack** → Wait until status is **CREATE_COMPLETE**  

---

## ご利用前に
事前に **CloudFormation** または **SageMaker** コンソールを使って、このサービスをデプロイしてください。

---

### オプション1 — SageMaker コンソールでのデプロイ
1. **AWS Marketplace** → サブスクライブ済み製品を開く → **構成** をクリック  
2. 起動方法で **SageMaker コンソール** を選択  
3. SageMaker コンソールで以下を設定:  
   - **モデル名**（任意の名前）  
   - **インスタンスタイプ**（例: `ml.m5.xlarge`）  
   - **エンドポイント名**（推論エンドポイントの名前）  
4. **エンドポイント作成** をクリック → ステータスが **InService** になるまで待機  

---

### オプション2 — CloudFormationでのデプロイ
1. **AWS Marketplace** → 製品ページ → **構成** → **CloudFormation** を選択  
2. 表示された **スタックの起動** リンクをクリック  
3. CloudFormation ページで以下を設定:  
   - **スタック名**（任意の名前）  
   - パラメータを確認（インスタンスタイプなど）  
4. **スタック作成** をクリック → ステータスが **CREATE_COMPLETE** になるまで待機  


## Data Prepartion

Sample Image：https://mlism-marketplace-documents.s3.ap-northeast-1.amazonaws.com/samples/gallery1.jpg

Sample PDF: https://www.soumu.go.jp/main_content/000975178.pdf

In [1]:
import requests

def download_image(url, filename):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)
        print(f"Image saved as {filename}")
    else:
        print(f"Failed to download image. Status code: {response.status_code}")

download_image(
    "https://mlism-marketplace-documents.s3.ap-northeast-1.amazonaws.com/samples/gallery1.jpg",
    "image.png"
)
download_image(
    "https://www.soumu.go.jp/main_content/000975178.pdf",
    "image.pdf"
)


Image saved as image.png
Image saved as image.pdf


In [2]:
import json
import boto3
from pprint import pprint

## Config Setup

In [ ]:
ENDPOINT_NAME = "" # Add your endpoint name,EX 
AWS_REGION = "" # Add your AWS_REGION
AWS_ACCESS_KEY_ID =  ""# Add your AWS_ACCESS_KEY_ID
AWS_SECRET_ACCESS_KEY = "" # Add your AWS_SECRET_ACCESS_KEY

In [ ]:
MFA_SERIAL = "" # arn of your mfa device
MFA_NUMBER = "" # mfa token

In [ ]:
base = boto3.Session(
    region_name = AWS_REGION,
    aws_access_key_id = AWS_ACCESS_KEY_ID,
    aws_secret_access_key=  AWS_SECRET_ACCESS_KEY
)

sts = base.client("sts")

In [ ]:
def with_mfa_session(base_sess, mfa_serial: str, token: str):
    """
    Create a session with MFA authentication.
    Args:
        base_sess: The base session. ベースセッション
        mfa_serial: The MFA serial. MFAシリアル
        token: The MFA token. MFAトークン
    Returns:
        A session with MFA authentication. MFA認証を使用したセッション
    """
    sts = base_sess.client("sts")
    resp = sts.get_session_token(SerialNumber=mfa_serial, TokenCode=token, DurationSeconds=43200)
    c = resp["Credentials"]
    return boto3.Session(
        region_name=base_sess.region_name,
        aws_access_key_id=c["AccessKeyId"],
        aws_secret_access_key=c["SecretAccessKey"],
        aws_session_token=c["SessionToken"],
    )

mfa_sess = with_mfa_session(base, MFA_SERIAL, MFA_NUMBER)
sm = mfa_sess.client("sagemaker")
status = sm.describe_endpoint(
    EndpointName=ENDPOINT_NAME
)["EndpointStatus"]
print(f"Endpoint status: {status}")

Endpoint status: InService


## Inference

#### Png

In [ ]:
rt = mfa_sess.client("sagemaker-runtime")
with open("./image.png", "rb") as f:
    response = rt.invoke_endpoint(
        EndpointName=ENDPOINT_NAME,
        ContentType="image/png", # Attention: Change the content type to the type of the file you are sending, support image/jpeg - JPEG images, image/png - PNG images, image/tiff - TIFF images, application/pdf - PDF files
        Body=f.read(),
    )

body_bytes = response["Body"].read()

result = json.loads(body_bytes)
pprint(result)

{'result': [{'figures': [{'box': [724, 60, 925, 162],
                          'caption': None,
                          'decode': None,
                          'direction': 'horizontal',
                          'order': 3,
                          'paragraphs': [{'box': [797, 111, 866, 146],
                                          'contents': '特集',
                                          'direction': 'horizontal',
                                          'indent_level': None,
                                          'order': 0,
                                          'role': None}],
                          'role': 'picture'},
                         {'box': [992, 1206, 1439, 1496],
                          'caption': None,
                          'decode': None,
                          'direction': 'horizontal',
                          'order': 6,
                          'paragraphs': [{'box': [1035, 1359, 1393, 1425],
                                       

#### PDF

In [ ]:
with open("./image.pdf", "rb") as f:
    response = rt.invoke_endpoint(
        EndpointName=ENDPOINT_NAME,
        ContentType="application/pdf", # Attention: ContentType must be application/pdf
        Body=f.read(),
    )

body_bytes = response["Body"].read()

result = json.loads(body_bytes)
pprint(result)

{'result': [{'figures': [{'box': [1475, 178, 1581, 348],
                          'caption': None,
                          'decode': None,
                          'direction': 'horizontal',
                          'order': 3,
                          'paragraphs': [{'box': [1471, 297, 1583, 345],
                                          'contents': 'MIC',
                                          'direction': 'horizontal',
                                          'indent_level': None,
                                          'order': 0,
                                          'role': None}],
                          'role': 'logo'},
                         {'box': [32, 605, 1624, 2281],
                          'caption': None,
                          'decode': None,
                          'direction': 'horizontal',
                          'order': 6,
                          'paragraphs': [{'box': [71, 637, 483, 683],
                                          '